<a href="https://colab.research.google.com/github/atorre35/atorre35.github.io/blob/main/week1_assignment1_pandas_exercises.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Week 1 Assignment - Pandas exercises

## Data

This assignment uses 2019 flight statistics from the United States Department of Transportation’s Bureau of Transportation Statistics (available [here](https://www.transtats.bts.gov/DL_SelectFields.asp?gnoyr_VQ=FMF&QO_fu146_anzr=Nv4%20Pn44vr45) and in this repository as `data/T100_MARKET_ALL_CARRIER.zip`). Each row contains information about a specific route for a given carrier in a given month (e.g., JFK &rarr; LAX on Delta Airlines in January). There are 321,409 rows and 41 columns. Note that you don't need to unzip the file to read it in with `pd.read_csv()`.

## Exercises

##### 1. Read in the data and convert the column names to lowercase to make them easier to work with.

In [ ]:
import pandas as pd

stats_df = pd.read_csv('T100_MARKET_ALL_CARRIER.csv')
stats_df.columns = stats_df.columns.str.lower()
stats_df.head()

,passengers,freight,mail,distance,unique_carrier,airline_id,unique_carrier_name,unique_carrier_entity,region,carrier,...,dest_state_nm,dest_country,dest_country_name,dest_wac,year,quarter,month,distance_group,class,data_source
0,0.0,53185.0,0.0,8165.0,EK,20392,Emirates,9678A,I,EK,...,Texas,US,United States,74,2019,1,3,17,G,IF
1,0.0,9002.0,0.0,6849.0,EK,20392,Emirates,9678A,I,EK,...,New York,US,United States,22,2019,1,3,14,G,IF
2,0.0,2220750.0,0.0,7247.0,EK,20392,Emirates,9678A,I,EK,...,Illinois,US,United States,41,2019,1,3,15,G,IF
3,0.0,1201490.0,0.0,8165.0,EK,20392,Emirates,9678A,I,EK,...,NaN,AE,United Arab Emirates,678,2019,1,3,17,G,IF
4,0.0,248642.0,0.0,6849.0,EK,20392,Emirates,9678A,I,EK,...,NaN,AE,United Arab Emirates,678,2019,1,3,14,G,IF


##### 2. What columns are in the data?

In [ ]:
stats_df.columns

Index(['passengers', 'freight', 'mail', 'distance', 'unique_carrier',
       'airline_id', 'unique_carrier_name', 'unique_carrier_entity', 'region',
       'carrier', 'carrier_name', 'carrier_group', 'carrier_group_new',
       'origin_airport_id', 'origin_airport_seq_id', 'origin_city_market_id',
       'origin', 'origin_city_name', 'origin_state_abr', 'origin_state_fips',
       'origin_state_nm', 'origin_country', 'origin_country_name',
       'origin_wac', 'dest_airport_id', 'dest_airport_seq_id',
       'dest_city_market_id', 'dest', 'dest_city_name', 'dest_state_abr',
       'dest_state_fips', 'dest_state_nm', 'dest_country', 'dest_country_name',
       'dest_wac', 'year', 'quarter', 'month', 'distance_group', 'class',
       'data_source'],
      dtype='object')

##### 3. How many distinct carrier names are in the dataset?

In [ ]:
len(stats_df['unique_carrier_name'].unique())
#318 distinct carrier names

318

##### 4. Calculate the totals of the `freight`, `mail`, and `passengers` columns for flights from the United Kingdom to the United States.

In [ ]:
uk_to_us = stats_df.loc[(stats_df['origin_country_name'] == 'United Kingdom') & (stats_df['dest_country_name'] == 'United States')]
totals_df = uk_to_us[['freight', 'mail', 'passengers']]

print(sum(totals_df['freight'])) #freight column total is 903,296,879
print(sum(totals_df['mail'])) #mail column total is 29,838,395
print(sum(totals_df['passengers'])) #passengers column total is 10,685,608

903296879.0
29838395.0
10685608.0


##### 5. Find the top 5 carriers for median route distance (i.e., for all origin-destination pairs that each carrier has, find the median distance after removing duplicates).

In [ ]:
stats_df2 = stats_df
stats_df2['origin_dest'] = (stats_df2['origin_country_name'] + ' -> ' + stats_df2['dest_country_name'])
stats_df2[['origin_country_name', 'dest_country_name', 'origin_dest']]
stats_df2['origin_dest'] = stats_df2['origin_dest'].drop_duplicates()
stats_df2 = stats_df2.dropna(subset=['origin_dest'])
stats_df2['distance'].median() #median distance after removing duplicates is 4,080.5

4080.5

##### 6. Find the total cargo transported (mail + freight) and mean distance traveled for the 10 carriers that transported the most cargo.

In [ ]:
stats_df['total_cargo'] = (stats_df['freight'] + stats_df['mail'])
top_cargo = stats_df[['freight', 'mail', 'distance', 'total_cargo']].sort_values(by='total_cargo', ascending=False).head(10)
print(sum(top_cargo['total_cargo'])) #total carg transported for 10 carriers with most cargo is 498,734,439
print(top_cargo['distance'].mean()) #mean distance traveled for 10 carriers with most cargo 2,535

498734439.0
2535.1


##### 7. Which 10 carriers flew the most passengers out of the United States to another country?

In [ ]:
us_orig = stats_df.loc[(stats_df['origin_country_name'] == 'United States') & (stats_df['dest_country_name'] != 'United States')]
top_pass = us_orig.groupby(by='unique_carrier_name')['passengers'].sum().sort_values(ascending=False)
print(top_pass.head(10))

unique_carrier_name
American Airlines Inc.       14867653.0
United Air Lines Inc.        14427923.0
Delta Air Lines Inc.         13054230.0
JetBlue Airways               4522492.0
British Airways Plc           3758945.0
Lufthansa German Airlines     3123611.0
Westjet                       2626600.0
Air Canada                    2540855.0
Southwest Airlines Co.        2146960.0
Virgin Atlantic Airways       2074735.0
Name: passengers, dtype: float64


##### 8. For each of the carriers found in *#7*, find the most popular destination country outside of the United States.

In [ ]:
top_df = top_pass.to_frame().head(10)
carrier_list = top_df.index.tolist()

top_carrier = us_orig[us_orig['unique_carrier_name'].isin(carrier_list)]
top_carrier['dest_country_name'].mode() #most popular destination country outside of US among top carriers is Canada

0    Canada
Name: dest_country_name, dtype: object


##### 9. For each of the carriers found in *#7*, find the total number of passengers flown on international flights to/from the destinations in *#8* or the United States. Note that this dataset only has data for flights with an origin and/or destination of the United States.

##### 10. Between which two cities were the most passengers flown? Make sure to account for both directions.

##### 11. Find the top 3 carriers for the pair of cities found in *#10* and calculate the percentage of passengers each accounted for.

##### 12. Find the percentage of international travel per country using total passengers on class F flights.

##### 13. Using a crosstab, find the percentage of total passengers on class F international flights between US cities and the countries found in *#12* that used the carriers found in *#11*.

##### 14. Create a pivot table showing the total passengers transported between cities in the United States and other countries by the carriers identified in *#7*. Select the top 10 US cities and top 10 international countries from the result.

##### 15: For the top 15 international countries, find the percentage of class F passengers traveling to/from the top 10 US cities for international travel (e.g. if only cities A, B, and C flew into Aruba, the sum of the Aruba row/column would be 1). Plot the result as a heatmap.